# Voxelize and preprocess MNist

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

data_path = "/home/ghandour/Dataset/MNIST"
class ThresholdTransform(object): ## https://stackoverflow.com/questions/65979207/applying-a-simple-transformation-to-get-a-binary-image-using-pytorch
  def __init__(self, thr_255):
    self.thr = thr_255 / 255.  # input threshold for [0..255] gray level, convert to [0..1]

  def __call__(self, x):
    return (x > self.thr).to(x.dtype)  # do not change the data type

transform = transforms.Compose([transforms.ToTensor(), ThresholdTransform(thr_255=80)])

dst_train = datasets.MNIST(data_path, train=True, download=True, transform=transform) # no augmentation
dst_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

In [10]:
min_pts = 9999
for i in range(len(dst_train)):
    elem = dst_train[i][0]
    occ_grid = torch.argwhere(elem==1)
    if min_pts > occ_grid.shape[0]:
        min_pts = occ_grid.shape[0]

print(min_pts)


25


In [16]:
class Mnist2D(datasets.MNIST):
    def __init__(self, root: str, train: bool = True, transform = None, target_transform= None, download: bool = False) -> None:
        super().__init__(root, train, transform, target_transform, download)

    def __getitem__(self, index: int):
        image, label = super().__getitem__(index)
        occ_grid = torch.argwhere(image==1)
        return occ_grid, label
    
    def __len__(self) -> int:
       return super().__len__()

In [17]:
threshold_ds = Mnist2D(root=data_path, train=True, transform=transform)
dataloader = DataLoader(threshold_ds)

In [23]:
import open3d as o3d
import meshplot as mp
import numpy as np


In [56]:
for batch in tqdm(dataloader):
    xyz = batch[0].squeeze().cpu().numpy() 
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    # mp.plot(np.array(p3d.points), shading={"point_size": 1})
    voxelgrid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=1)
    filename = "/home/ghandour/"+str(batch[1].item())+"2.ply"
    print()
    # o3d.io.write_voxel_grid(filename, voxelgrid, write_ascii = False, compressed = False, print_progress = False)
    o3d.io.write_triangle_mesh(filename, voxelgrid.TriangleMesh)

    break

  0%|          | 0/60000 [00:00<?, ?it/s]

TypeError: write_triangle_mesh(): incompatible function arguments. The following argument types are supported:
    1. (filename: str, mesh: open3d.cuda.pybind.geometry.TriangleMesh, write_ascii: bool = False, compressed: bool = False, write_vertex_normals: bool = True, write_vertex_colors: bool = True, write_triangle_uvs: bool = True, print_progress: bool = False) -> bool

Invoked with: '/home/ghandour/52.ply', <Type.TriangleMesh: 6>